In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from itertools import combinations
from collections import defaultdict

In [2]:
data_fp = '/Users/JakeCanfield/Documents/Trauma_Surgery_Research/Data/Raw_data/PUF AY %i/CSV/'
years = range(2007, 2017)
thoracotomy_codes = [34.02]
sternotomy_codes = ['0P800ZZ']

mechanism_code_dict = {1:'Cut/pierce', 2:'Drowning/submersion', 3:'Fall', 4:'Fire/flame', 5:'Hot object/substance', 6:'Firearm', 7:'Machinery', 8:'MVT Occupant', 9:'MVT Motorcyclist', 10:'MVT Pedal cyclist', 11:'MVT Pedestrian', 12:'MVT Unspecified', 13:'MVT Other', 14:'Pedal cyclist, other', 15:'Pedestrian, other', 16:'Transport, other', 17:'Natural/environmental,  Bites and stings', 18:'Natural/environmental,  Other', 19:'Overexertion', 20:'Poisoning', 21:'Struck by, against', 22:'Suffocation', 23:'Other specified and classifiable', 24:'Other specified, not elsewhere classifiable', 25:'Unspecified', 26:'Adverse effects, medical care', 27:'Adverse effects, drugs'} # As noted in PUF dictionary
trauma_type_code_dict = {1:'Blunt', 2:'Penetrating', 3:'Burn', 4:'Other/unspecified', 9:'Activity Code - Not Valid as a Primary E-Code'} # As noted in PUF Dictionary
sex_code_dict = {1:'Male', 2:'Female'}
eddischarge_code_dict = {1: 'Floor bed (general admission, non-specialty unit bed)', 2: 'Observation unit (unit that provides < 24 hour stays)', 3: 'Telemetry/step-down unit (less acuity than ICU)', 4: 'Home with services', 5: 'Deceased/expired', 6: 'Other (jail, institutional care, mental health, etc.)', 7: 'Operating Room', 8: 'Intensive Care Unit (ICU)', 9: 'Home without services', 10: 'Left against medical advice', 11: 'Transferred to another hospital'}
hospdischarge_disposition_code_dict = {1: 'Discharged/Transferred to a short-term general hospital for inpatient care', 2: 'Discharged/Transferred to an Intermediate Care Facility (ICF)', 3: 'Discharged/Transferred to home under care of organized home health service', 4: 'Left against medical advice or discontinued care', 5: 'Deceased/Expired', 6: 'Discharged to home or self-care (routine discharge)', 7: 'Discharged/Transferred to Skilled Nursing Facility (SNF)', 8: 'Discharged/Transferred to hospice care', 10: 'Discharged/Transferred to court/law enforcement', 11: 'Discharged/Transferred to inpatient rehab or designated unit', 12: 'Discharged/Transferred to Long Term Care Hospital (LTCH)', 13: 'Discharged/Transferred to a psychiatric hospital or psychiatric distinct part unit of a hospital', 14: 'Discharged/Transferred to another type of institution not defined elsewhere'}

In [47]:
display(pd.read_csv(data_fp%2016+'PUF_PM.csv').columns.tolist())

['inc_key',
 'TBI_HIGHEST_TOTAL_GCS',
 'TBI_GCS_MOTOR',
 'VTE_PROPHYLAXIS_TYPE_CODE',
 'VTE_PROPHYLAXIS_TYPE_DESC',
 'LOWEST_SBP',
 'TRANS_BLOOD_4HOURS',
 'TRANS_PLASMA_4HOURS',
 'TRANS_PLATELETS_4HOURS',
 'CRYOPRECIPITATE_4HOURS',
 'TRANS_BLOOD_24HOURS',
 'TRANS_PLASMA_24HOURS',
 'TRANS_PLATELETS_24HOURS',
 'CRYOPRECIPITATE_24HOURS',
 'ANGIOGRAPHY_CODE',
 'ANGIOGRAPHY_DESC',
 'HEMORRHAGE_CTRL_STYPE_CODE',
 'HEMORRHAGE_CTRL_STYPE_DESC',
 'WITHDRAWAL_OF_CARE_CODE',
 'WITHDRAWAL_OF_CARE_DESC',
 'TRANS_BLOOD_MEASURE_CODE',
 'TRANS_BLOOD_MEASURE_DESC',
 'TRANS_BLOOD_CONV',
 'TRANS_PLASMA_MEASURE_CODE',
 'TRANS_PLASMA_MEASURE_DESC',
 'TRANS_PLASMA_CONV',
 'TRANS_PLATELETS_MEASURE_CODE',
 'TRANS_PLATELETS_MEASURE_DESC',
 'TRANS_PLATELETS_CONV',
 'CRYOPRECIPITATE_MEASURE_CODE',
 'CRYOPRECIPITATE_MEASURE_DESC',
 'CRYOPRECIPITATE_CONV',
 'PUPILLARY_RESPONSE_CODE',
 'PUPILLARY_RESPONSE_DESC',
 'MIDLINE_SHIFT_CODE',
 'MIDLINE_SHIFT_DESC',
 'CerebralMonitorMins',
 'CerebralMonitorDays',
 'VTEMins'

In [75]:
for year in years:
    print(year)
    if year in range(2010, 2017, 1):
        PCODE_df = pd.read_csv(data_fp%year + 'PUF_PCODE.csv').sort_values('INC_KEY')
        PCODE_df = PCODE_df.rename(columns={'HOURTOPROC': 'HOURTOPRO'})
    else:
        PCODE_df = pd.read_csv(data_fp%year + 'PUF_PCODE.csv').sort_values('INC_KEY')
    PCODEDES_df = pd.read_csv(data_fp%year + 'PUF_PCODEDES.csv')
    DISCHARGE_df = pd.read_csv(data_fp%year + 'PUF_DISCHARGE.csv', usecols=['INC_KEY', 'HOSPDISP', 'LOSMIN', 'LOSDAYS', 'ICUDAYS', 'VENTDAYS']).sort_values('INC_KEY')
    ECODE_df = pd.read_csv(data_fp%year + 'PUF_ECODE.csv', usecols=['INC_KEY', 'ECODE']).sort_values('INC_KEY')
    ECODEDES_df = pd.read_csv(data_fp%year + 'PUF_ECODEDES.csv', usecols=['INJTYPE', 'MECHANISM', 'ECODE', 'ECODEDES'])
    DEMO_df = pd.read_csv(data_fp%year + 'PUF_DEMO.csv', usecols=['INC_KEY', 'GENDER', 'AGE', 'RACE1', 'RACE2', 'ETHNIC']).sort_values('INC_KEY')
    if year in range(2013, 2017, 1):
        PM_df = pd.read_csv(data_fp%year + 'PUF_PM.csv', usecols=['inc_key', 'HEMORRHAGE_CTRL_STYPE_CODE', 'HEMORRHAGE_CTRL_STYPE_DESC', 'HemorrhageCtrlMins', 'HemorrhageCtrlDays']).sort_values('inc_key')
        PM_df.columns = map(str.upper, PM_df.columns)
    else:
        pass
    
    if year in range(2007, 2011, 1):
        ED_df = pd.read_csv(data_fp%year + 'PUF_ED.csv', usecols=['INC_KEY', 'EDDISP', 'EDDEATH']).sort_values('INC_KEY')
    elif year in range(2011, 2017, 1):
        ED_df = pd.read_csv(data_fp%year + 'PUF_ED.csv', usecols=['INC_KEY', 'EDDISP', 'SIGNSOFLIFE']).sort_values('INC_KEY')
        ED_df = ED_df.rename(columns={'SIGNSOFLIFE':'EDDEATH'})
    else:
        pass
    
    # Force column headers to be all upper case and fill in missing year data
    
    PCODE_df.columns = map(str.upper, PCODE_df.columns)
    ED_df.columns = map(str.upper, ED_df.columns)
    PCODEDES_df.columns = map(str.upper, PCODEDES_df.columns)
    PCODE_df['YOPROC'] = PCODE_df['YOPROC'].fillna(year)
    
    PCODE_df['PROCEDUREMINS'] = PCODE_df['HOURTOPRO']*60
    
    PCODE_df = PCODE_df.loc[:, ['INC_KEY', 'PCODE', 'DAYTOPROC', 'PROCEDUREMINS', 'HOURTOPRO']]
    
    ECODE_LOOKUP_df = ECODEDES_df.loc[:, ['ECODE', 'ECODEDES', 'MECHANISM', 'INJTYPE']]
    
    print(np.shape(PCODE_df))
    
    thoracotomy_list = PCODE_df.loc[PCODE_df['PCODE'].isin(thoracotomy_codes)]
    
    print(np.shape(thoracotomy_list))

    if year in range(2013, 2017, 1):
        DCS_list = PM_df.loc[PM_df['HEMORRHAGE_CTRL_STYPE_CODE'] == 3.0]
        thoracotomy_keys = np.unique(thoracotomy_list['INC_KEY'].tolist() + DCS_list['INC_KEY'].tolist())
        print(np.shape(thoracotomy_keys))
    else:
        thoracotomy_keys = np.unique(thoracotomy_list['INC_KEY'].tolist())
        print(np.shape(thoracotomy_keys))

    
    PCODE_df = PCODE_df.loc[PCODE_df['INC_KEY'].isin(thoracotomy_keys)]
    ECODE_df = ECODE_df.loc[ECODE_df['INC_KEY'].isin(thoracotomy_keys)]
    DEMO_df = DEMO_df.loc[DEMO_df['INC_KEY'].isin(thoracotomy_keys)]
    ED_df = ED_df.loc[ED_df['INC_KEY'].isin(thoracotomy_keys)]
    DISCHARGE_df = DISCHARGE_df.loc[DISCHARGE_df['INC_KEY'].isin(thoracotomy_keys)]
    
    print(np.shape(ECODE_df))
    
    procedure_dict = dict(zip(PCODEDES_df['PCODE'], PCODEDES_df['PCODEDESCR'])) # create dictionary
    PCODE_df['PCODE'] = PCODE_df['PCODE'].replace(procedure_dict) # implement dictionary
    
    ecode_dict = dict(zip(ECODEDES_df['ECODE'], ECODEDES_df['ECODEDES'])) # create dictionary
    ECODE_df['ECODE'] = ECODE_df['ECODE'].replace(ecode_dict) # implement dictionary
    
    print(np.shape(ECODE_df))
    
    mechanism_dict = dict(zip(ECODEDES_df['ECODEDES'], ECODEDES_df['MECHANISM'])) # create dictionary
    ECODE_df['MECHANISM'] = ECODE_df['ECODE'].map(mechanism_dict) # implement dictionary
    
    print(np.shape(ECODE_df))
    
    traumatype_dict = dict(zip(ECODEDES_df['ECODEDES'], ECODE_LOOKUP_df['INJTYPE'])) # create dictionary
    ECODE_df['INJTYPE'] = ECODE_df['ECODE'].map(traumatype_dict) # implement dictionary
    
    print(np.shape(ECODE_df))
    
    ECODE_df['MECHANISM'] = ECODE_df['MECHANISM'].replace(mechanism_code_dict) # implement dictionary, may not need
    
    print(np.shape(ECODE_df))
    
    ECODE_df['INJTYPE'] = ECODE_df['INJTYPE'].replace(trauma_type_code_dict) # implement dictionary, may not need
    
    print(np.shape(ECODE_df))
    
    DEMO_df['GENDER'] = DEMO_df['GENDER'].replace(sex_code_dict) # May not need
    
    print(np.shape(DEMO_df))
    
    ED_df['EDDISP'] = ED_df['EDDISP'].replace(eddischarge_code_dict) # implement dictionary
    
    print(np.shape(ED_df))
    
    DISCHARGE_df['HOSPDISP'] = DISCHARGE_df['HOSPDISP'].replace(hospdischarge_disposition_code_dict) # implement dictionary
    
    DCS_df = PM_df.loc[PM_df['INC_KEY'].isin(thoracotomy_keys)]
    
    print(np.shape(DISCHARGE_df))
    
    display(PCODE_df.head())
    print(np.shape(PCODE_df))
    display(ECODE_df.head())
    print(np.shape(ECODE_df))
    display(DEMO_df.head())
    print(np.shape(DEMO_df))
    display(ED_df.head())
    print(np.shape(ED_df))
    display(DISCHARGE_df.head())
    print(np.shape(DISCHARGE_df))
    
    if year in range(2013, 2017, 1):
        TRAUMA_df = pd.merge(pd.merge(pd.merge(pd.merge(ECODE_df, DEMO_df, on='INC_KEY', how='left'), ED_df, on='INC_KEY', how='left'), DISCHARGE_df, on='INC_KEY', how='left'), DCS_df, on='INC_KEY', how='left')
    elif year in range(2007, 2013, 1):
        ECODE_df = ECODE_df.assign(**{'HEMORRHAGE_CTRL_STYPE_CODE': np.nan, 'HEMORRHAGE_CTRL_STYPE_DESC': np.nan, 'HEMORRHAGECTRLMINS': np.nan, 'HEMORRHAGECTRLDAYS': np.nan})
        TRAUMA_df = pd.merge(pd.merge(pd.merge(ECODE_df, DEMO_df, on='INC_KEY', how='left'), ED_df, on='INC_KEY', how='left'), DISCHARGE_df, on='INC_KEY', how='left')
    TRAUMA_df = TRAUMA_df.rename(columns={'ECODE':'PRIMARYECODEICD10', 'INJTYPE':'TRAUMATYPE','GENDER':'SEX', 'EDDISP':'EDDISCHARGEDISPOSITION', 'HOSPDISP':'HOSPDISCHARGEDISPOSITION', 'HEMORRHAGECTRLMINS':'HMRRHGCTRLSURGMINS', 'HEMORRHAGECTRLDAYS':'HMRRHGCTRLSURGDAYS', 'AGE':'AGEYEARS', 'HEMORRHAGE_CTRL_STYPE_CODE': 'HMRRHGCTRLSURGTYPE'})
    
    PCODE_df = PCODE_df.rename(columns={'PCODE':'ICDPROCEDURECODE', 'DAYTOPROC':'PROCEDUREDAYS'})
    
    TRAUMA_df = TRAUMA_df.loc[TRAUMA_df['TRAUMATYPE'].isin(['Blunt', 'Penetrating'])]
    
    TRAUMA_df.to_csv('/Users/JakeCanfield/Documents/Trauma_Surgery_Research/data/Combined_data/newTRAUMA_df_%i.csv'%year, index=False)
    PCODE_df.to_csv('/Users/JakeCanfield/Documents/Trauma_Surgery_Research/data/Combined_data/newICDPROCEDURE_df_%i.csv'%year, index=False)
          
    display(PCODE_df.head())
    display(TRAUMA_df.head())

2007
(1640995, 5)
(1881, 5)
(1812,)
(1812, 2)
(1812, 2)
(1812, 3)
(1812, 4)
(1812, 4)
(1812, 4)
(1812, 6)
(1812, 3)
(1812, 6)


,INC_KEY,PCODE,DAYTOPROC,PROCEDUREMINS,HOURTOPRO
1232920,402,Insertion of intercostal catheter for drainage,1,180,3
617590,402,Insertion of endotracheal tube,1,180,3
22517,402,Exploratory thoracotomy,1,180,3
22519,402,Other miscellaneous procedures,1,600,10
1028076,402,Other repair of diaphragm,1,180,3


(17486, 5)


,INC_KEY,ECODE,MECHANISM,INJTYPE
445612,402,Oth and Unspec Fall - Unspec,Fall,Blunt
417206,839,Oth and Unspec Fall - Unspec,Fall,Blunt
5964,943,"MVA Traffic, Loss Control-No Collision - Drive...",MVT Occupant,Blunt
4411,1013,"Oth MVA Traffic, Collision w/ MV - Driver of M...",MVT Occupant,Blunt
4997,1088,Assault by Cutting and Piercing Instrument,Cut/pierce,Penetrating


(1812, 4)


...

(2592, 3)


,INC_KEY,HOSPDISP,LOSMIN,LOSDAYS,ICUDAYS,VENTDAYS
394,160000396,Deceased/Expired,86,1,-1,1
461,160000463,Deceased/Expired,9075,7,7,7
756,160000758,Deceased/Expired,39,1,-1,1
984,160000986,Deceased/Expired,11093,8,9,9
993,160000995,Not Applicable BIU 1,18,1,-1,-1


(2592, 6)


,INC_KEY,ICDPROCEDURECODE,PROCEDUREDAYS,PROCEDUREMINS,HOURTOPRO
1867031,160000396,Insertion of endotracheal tube,NaN,NaN,NaN
1901886,160000396,"Venous catheterization, not elsewhere classified",NaN,NaN,NaN
1869652,160000396,Exploratory thoracotomy,NaN,NaN,NaN
1867030,160000396,Other electric countershock of heart,NaN,NaN,NaN
1896184,160000396,Exploratory thoracotomy,1.0,40.02,0.667


,INC_KEY,PRIMARYECODEICD10,MECHANISM,TRAUMATYPE,AGEYEARS,SEX,RACE1,RACE2,ETHNIC,EDDISCHARGEDISPOSITION,EDDEATH,HOSPDISCHARGEDISPOSITION,LOSMIN,LOSDAYS,ICUDAYS,VENTDAYS,HMRRHGCTRLSURGTYPE,HEMORRHAGE_CTRL_STYPE_DESC,HMRRHGCTRLSURGMINS,HMRRHGCTRLSURGDAYS
0,160000396,"N-traffic Accident, Oth Off-Road MV - Driver o...","Transport, other",Blunt,23,Female,Other Race,Not Known/Not Recorded BIU 2,Hispanic or Latino,Operating Room,Arrived with signs of life,Deceased/Expired,86,1,-1,1,3.0,Thoracotomy,40.0,1.0
1,160000463,Assault by Cutting and Piercing Instrument,Cut/pierce,Penetrating,33,Male,Black or African American,Not Known/Not Recorded BIU 2,Not Hispanic or Latino,Operating Room,Arrived with signs of life,Deceased/Expired,9075,7,7,7,3.0,Thoracotomy,26.0,1.0
2,160000758,"Oth MVA Traffic, Collision w/ MV - Passenger i...",MVT Occupant,Blunt,46,Female,White,Not Known/Not Recorded BIU 2,Not Hispanic or Latino,Operating Room,Arrived with NO signs of life,Deceased/Expired,39,1,-1,1,3.0,Thoracotomy,3.0,1.0
5,160001001,Assault by Cutting and Piercing Instrument,Cut/pierce,Penetrating,22,Male,Other Race,Not Known/Not Recorded BIU 2,Hispanic or Latino,Deceased/Expired,Arrived with NO signs of life,Not Applicable BIU 1,15,1,-1,1,-1.0,Not Applicable BIU 1,NaN,NaN
6,160001246,Assault by Firearms/Explosives - Handgun,Firearm,Penetrating,24,Male,Black or African American,Not Known/Not Recorded BIU 2,Not Hispanic or Latino,Deceased/Expired,Arrived with signs of life,Not Applicable BIU 1,31,1,-1,-1,1.0,None,NaN,NaN


In [70]:
print(TRAUMA_df.columns.tolist())
print()
print(PCODE_df.columns.tolist())

['INC_KEY', 'ECODE', 'MECHANISM', 'INJTYPE', 'AGE', 'GENDER', 'RACE1', 'RACE2', 'ETHNIC', 'SEX', 'EDDISP', 'SIGNSOFLIFE', 'HOSPDISP', 'LOSMIN', 'LOSDAYS', 'ICUDAYS', 'VENTDAYS', 'HEMORRHAGE_CTRL_STYPE_CODE', 'HEMORRHAGE_CTRL_STYPE_DESC', 'HEMORRHAGECTRLMINS', 'HEMORRHAGECTRLDAYS']

['INC_KEY', 'PCODE', 'DAYTOPROC', 'PROCEDUREMINS', 'HOURTOPRO']
